# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time
import torch.optim as optim
from torch.autograd import Variable
import sys

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(120, 84),
            nn.ReLU()
        )
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

model = SimpleNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Next, we can start to train and evaluate!

In [4]:
# train and evaluate
train_accuracy=[]
test_accuracy=[]
for epoch in range(NUM_EPOCHS):
    train_correct = 0
    train_total = 0
    for images, labels in tqdm(train_loader):
        images, labels = Variable(images).to(device), Variable(labels).to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    model.eval()
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = Variable(images).to(device), Variable(labels).to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
    train_accuracy.append(100.*train_correct/train_total)
    test_accuracy.append(100.*test_correct/test_total)
    print("Epoch{}: Training accuracy: {:.3f}%, Testing accuracy: {:.3f}%\n".format(epoch,(100.*train_correct/train_total),(100.*test_correct/test_total)))
    sys.stdout.flush()

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:01<00:00, 53.18it/s]

Epoch0: Training accuracy: 90.363%, Testing accuracy: 97.596%




100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:01<00:00, 51.64it/s]

Epoch1: Training accuracy: 97.404%, Testing accuracy: 98.377%




100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:01<00:00, 52.34it/s]

...

Epoch7: Training accuracy: 99.289%, Testing accuracy: 99.099%




100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:01<00:00, 48.94it/s]

Epoch8: Training accuracy: 99.332%, Testing accuracy: 99.028%




100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:01<00:00, 47.77it/s]

Epoch9: Training accuracy: 99.447%, Testing accuracy: 98.668%



#### Q5:
Please print the training and testing accuracy.

In [5]:
for epoch in range(NUM_EPOCHS):
    print("Epoch{}: Training accuracy: {:.3f}%, Testing accuracy: {:.3f}%\n".format(epoch,train_accuracy[epoch],test_accuracy[epoch]))

Epoch0: Training accuracy: 90.363%, Testing accuracy: 97.596%

Epoch1: Training accuracy: 97.404%, Testing accuracy: 98.377%

Epoch2: Training accuracy: 98.137%, Testing accuracy: 98.608%

Epoch3: Training accuracy: 98.589%, Testing accuracy: 98.738%

Epoch4: Training accuracy: 98.821%, Testing accuracy: 98.928%

Epoch5: Training accuracy: 99.017%, Testing accuracy: 98.878%

Epoch6: Training accuracy: 99.229%, Testing accuracy: 98.838%

Epoch7: Training accuracy: 99.289%, Testing accuracy: 99.099%

Epoch8: Training accuracy: 99.332%, Testing accuracy: 99.028%

Epoch9: Training accuracy: 99.447%, Testing accuracy: 98.668%

